In [9]:
import sys
sys.path.append('/host/d/Github')
import os
import numpy as np
import pandas as pd
import nibabel as nb
import shutil
import SimpleITK as sitk
import Aorta_segmentation_ZC.functions_collection as ff
import Aorta_segmentation_ZC.Data_processing as Data_processing

In [3]:
main_path = '/host/d/Data/CTA/original_data/TAA_peking'
data_path = '/host/d/Data/CTA/TAA_peking'

### re-organize our data

In [15]:
files = ff.find_all_target_files(['*.nrrd'],os.path.join(data_path, 'CTA_cropped'))
print('Total files found:', len(files))

Total files found: 93


In [ ]:
for i in range(0, len(files)):
    filename = os.path.basename(files[i])

    # file name is like X-t0-CTA.nrrd
    patient_id = filename.split('-')[0]
    patient_tf = filename.split('-')[1]  

    # image original path
    image_original_path = os.path.join(data_path, 'CTA',patient_id+'-'+patient_tf+'-CTA.nrrd')
    if not os.path.exists(image_original_path):
        print('Image file not found:', image_original_path)
        # continue
    else:
        data, affine,_,_,_ = ff.load_nrrd(image_original_path)
        data = np.rollaxis(data,0,3)
        data = np.flip(np.flip(np.rollaxis(data,0,2),axis = 1),axis = 0)
        image_original_path_new = os.path.join(main_path, patient_id, patient_tf, 'image.nii.gz')
        ff.make_folder([os.path.join(main_path,patient_id), os.path.join(main_path,patient_id,patient_tf)])
        nii_image = nb.Nifti1Image(data, affine)
        nb.save(nii_image, image_original_path_new)

    # image cropped path
    image_cropped_path = files[i]
    data, affine,_,_,_ = ff.load_nrrd(image_cropped_path)
    data = np.rollaxis(data,0,3)
    data = np.flip(np.flip(np.rollaxis(data,0,2),axis = 1),axis = 0)
    image_cropped_path_new = os.path.join(main_path, patient_id, patient_tf, 'image_cropped.nii.gz')
    nii_image = nb.Nifti1Image(data, affine)
    nb.save(nii_image, image_cropped_path_new)

    # mask original path
    mask_original_path = os.path.join(data_path, 'labelmap',patient_id+'-'+patient_tf+'_label.nrrd')
    if not os.path.exists(mask_original_path):
        print('Mask file not found:', mask_original_path)
        # continue
    else:
        data, affine,_,_,_ = ff.load_nrrd(mask_original_path)
        data = np.rollaxis(data,0,3)
        data = np.flip(np.flip(np.rollaxis(data,0,2),axis = 1),axis = 0)
        mask_original_path_new = os.path.join(main_path, patient_id, patient_tf, 'seg.nii.gz')
        nii_image = nb.Nifti1Image(data, affine)
        nb.save(nii_image, mask_original_path_new)

    # mask cropped path
    mask_cropped_path = os.path.join(data_path, 'labelmap_cropped',patient_id+'-'+patient_tf+'_label.nrrd')
    if not os.path.exists(mask_cropped_path):
        print('Cropped mask file not found:', mask_cropped_path)
        # continue
    else:
        data, affine,_,_,_ = ff.load_nrrd(mask_cropped_path)
        data = np.rollaxis(data,0,3)
        data = np.flip(np.flip(np.rollaxis(data,0,2),axis = 1),axis = 0)
        mask_cropped_path_new = os.path.join(main_path, patient_id, patient_tf, 'seg_cropped.nii.gz')
        nii_image = nb.Nifti1Image(data, affine)
        nb.save(nii_image, mask_cropped_path_new)


### print out info

In [58]:
main_path = '/host/d/Data/CTA/original_data/'
patient_list = ff.sort_timeframe(ff.find_all_target_files(['TAA_peking/*'], main_path),0)
print('Number of cases found:', len(patient_list))

Number of cases found: 47


In [ ]:
results = []
for j in range(0, len(patient_list)):
    case_list = ff.find_all_target_files(['*'], patient_list[j])
    for i in range(0, len(case_list)):
        patient_id = os.path.basename(case_list[i])
        patient_subclass = os.path.basename(os.path.dirname(case_list[i]))
        patient_class = os.path.basename(os.path.dirname(os.path.dirname(case_list[i])))
        # print('patient class:', patient_class, ' subclass:', patint_subclass, ' id:', patient_id)
        image_file = os.path.join(case_list[i], 'seg_cropped.nii.gz')

        img_nii = nb.load(image_file)
        spacing = img_nii.header.get_zooms()
        spacing_x, spacing_y, spacing_z = spacing
        shape = img_nii.shape
        shape_x, shape_y, shape_z = shape

        seg_data = img_nii.get_fdata()
        if np.sum(seg_data) == 0:
            print('Warning: no segmentation found for case:', patient_subclass, patient_id)
        else:
            results.append([patient_class, patient_subclass, patient_id, shape_x, shape_y, shape_z, spacing_x, spacing_y, spacing_z])

        df = pd.DataFrame(results, columns=['class', 'subclass', 'id', 'shape_x', 'shape_y', 'shape_z', 'spacing_x', 'spacing_y', 'spacing_z'])
        df.to_excel(os.path.join(os.path.dirname(os.path.dirname(main_path)), 'Patient_lists/original_data_info_TAA.xlsx'))


### crop and resample data

In [28]:
main_path = '/host/d/Data/CTA/original_data'

In [31]:
patient_list = pd.read_excel(os.path.join(os.path.dirname(main_path), 'Patient_lists/original_data_info_TAA.xlsx'))
process_path = '/host/d/Data/CTA/processed_data'

for i in range(0, patient_list.shape[0]):
    row = patient_list.iloc[i]
    spacing_z = row['spacing_z']
    spacing_x = row['spacing_x']
    patient_id = str(row['id'])
    patient_class = str(row['class'])
    patient_subclass = str(row['subclass'])

    img_file = os.path.join(main_path, patient_class, patient_subclass, patient_id, 'image_cropped.nii.gz')
    seg_file = os.path.join(main_path, patient_class, patient_subclass, patient_id, 'seg_cropped.nii.gz')

    # resample to 1.5mm
    # do image first
    img_nifti = nb.load(img_file)
    img = img_nifti.get_fdata()
    img_resampled_nifti = Data_processing.resample_nifti(img_nifti, order = 3, mode = 'nearest', cval = np.min(img), in_plane_resolution_mm = 1.5, slice_thickness_mm = 1.5, number_of_slices = None)

    header = img_resampled_nifti.header
    affine = img_resampled_nifti.affine
    img_resampled_data = img_resampled_nifti.get_fdata()
    img_resampled_data = np.round(img_resampled_data).astype(np.int16)
    img_resampled_nifti = nb.Nifti1Image(img_resampled_data, affine, header)
    ff.make_folder([os.path.join(process_path, patient_class), os.path.join(process_path, patient_class, patient_subclass), os.path.join(process_path, patient_class, patient_subclass, patient_id)])
    nb.save(img_resampled_nifti, os.path.join(process_path, patient_class, patient_subclass, patient_id, 'img_resampled.nii.gz'))

    # do segmentation
    seg_nifti = nb.load(seg_file)
    seg = seg_nifti.get_fdata()
    seg_resampled_nifti = Data_processing.resample_nifti(seg_nifti, order = 0, mode = 'nearest', cval = 0, in_plane_resolution_mm = 1.5, slice_thickness_mm = 1.5, number_of_slices = img_resampled_data.shape[2])
    header = seg_resampled_nifti.header
    affine = seg_resampled_nifti.affine
    seg_resampled_data = seg_resampled_nifti.get_fdata()
    seg_resampled_data = np.round(seg_resampled_data).astype(np.int16)
    seg_resampled_nifti = nb.Nifti1Image(seg_resampled_data, affine, header)
    nb.save(seg_resampled_nifti, os.path.join(process_path, patient_class, patient_subclass, patient_id, 'seg_resampled.nii.gz'))   

### find out which cases have empty segmentation labels

In [51]:
case_list = ff.find_all_target_files(['TAA_peking/*/*'], process_path)
print('Number of cases found:', len(case_list))
for i in range(0, len(case_list)):
    seg_file = os.path.join(case_list[i], 'seg_resampled.nii.gz')
    seg_data = nb.load(seg_file).get_fdata()
    if np.sum(seg_data) == 0:
        print('No segmentation found for case:', case_list[i])

Number of cases found: 90


### print out resampled data info

In [63]:
patient_list = ff.sort_timeframe(ff.find_all_target_files(['TAA_peking/*'], process_path),0)
print('Number of cases found:', len(patient_list))

Number of cases found: 47


In [64]:
results = []
for j in range(0, len(patient_list)):
    case_list = ff.find_all_target_files(['*'], patient_list[j])
    for i in range(0, len(case_list)):
        patient_id = os.path.basename(case_list[i])
        patint_subclass = os.path.basename(os.path.dirname(case_list[i]))
        patient_class = os.path.basename(os.path.dirname(os.path.dirname(case_list[i])))
        print('patient class:', patient_class, ' subclass:', patint_subclass, ' id:', patient_id)
        image_file = os.path.join(case_list[i], 'img_resampled.nii.gz')

        img_nii = nb.load(image_file)
        spacing = img_nii.header.get_zooms()
        spacing_x, spacing_y, spacing_z = spacing
        shape = img_nii.shape
        shape_x, shape_y, shape_z = shape

        seg_file = os.path.join(case_list[i], 'seg_resampled.nii.gz')
        seg_nii = nb.load(seg_file).get_fdata()
        slices_with_aorta = np.where(np.sum(np.sum(seg_nii, axis = 0), axis = 0)>0)[0]
        aorta_slice_start, aorta_slice_end = slices_with_aorta[0], slices_with_aorta[-1]

        results.append([patient_class, patint_subclass, patient_id, shape_x, shape_y, shape_z, spacing_x, spacing_y, spacing_z, aorta_slice_start, aorta_slice_end])
        df = pd.DataFrame(results, columns=['class', 'subclass', 'id', 'shape_x', 'shape_y', 'shape_z', 'spacing_x', 'spacing_y', 'spacing_z', 'aorta_slice_start', 'aorta_slice_end'])
        df.to_excel(os.path.join('/host/d/Data/CTA','Patient_lists/resampled_data_info_TAA.xlsx'))

patient class: TAA_peking  subclass: 9  id: t0
patient class: TAA_peking  subclass: 9  id: t1
patient class: TAA_peking  subclass: 27  id: t0
patient class: TAA_peking  subclass: 27  id: t1
patient class: TAA_peking  subclass: 38  id: t0
patient class: TAA_peking  subclass: 38  id: t1
patient class: TAA_peking  subclass: 73  id: t0
patient class: TAA_peking  subclass: 73  id: t1
patient class: TAA_peking  subclass: 122  id: t0
patient class: TAA_peking  subclass: 122  id: t1
patient class: TAA_peking  subclass: 165  id: t0
patient class: TAA_peking  subclass: 165  id: t1
patient class: TAA_peking  subclass: 193  id: t0
patient class: TAA_peking  subclass: 193  id: t1
patient class: TAA_peking  subclass: 198  id: t0
patient class: TAA_peking  subclass: 198  id: t1
patient class: TAA_peking  subclass: 212  id: t0
patient class: TAA_peking  subclass: 212  id: t1
patient class: TAA_peking  subclass: 218  id: t0
patient class: TAA_peking  subclass: 218  id: t1
patient class: TAA_peking  sub